# Imports and load input files

In [1]:
from surprise import Reader, Dataset
import pandas as pd
import numpy as np
from utils import *
from pathlib import Path
import gc
from datetime import datetime
from sklearn.metrics import log_loss
import blaze

d:\Tools\Miniconda3\envs\py_3_6_5_keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
d:\Tools\Miniconda3\envs\py_3_6_5_keras\lib\site-packages\odo\backends\pandas.py:102: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
# csv_file = Path('Data/surprise_subset_train.csv')
# csv_data = blaze.Data('Data/surprise_train.csv')
# print('Loading dataframe.....')
# query = csv_data[csv_data['Status_Dt'].isin(['2015-05-28', '2016-04-28', '2016-05-28'])]
# train = pd.DataFrame.from_records(query, columns=query.fields)

# print('Saving dataframe to file.....')
# surprise_cols = ['Cust_ID', 'Product_ID', 'Rating', 'Status_Dt']
# train = pd.read_csv(csv_file, usecols = surprise_cols)
# train.drop(train[train.Rating.isnull()].index, inplace=True)
# train.to_csv('Data/surprise_subset_train.csv', index=False)
# train.head()

Loading dataframe.....


d:\Tools\Miniconda3\envs\py_3_6_5_keras\lib\site-packages\blaze\compute\pandas.py:792: UserWarning: Counting the elements of a dask object can be slow.
  warnings.warn("Counting the elements of a dask object can be slow.")


Saving dataframe to file.....


In [5]:
# train.drop(train[train.Status_Dt == '2015-04-28'].index, inplace=True)

In [7]:
# train.to_csv('Data/surprise_subset_train.csv', index=False)
# train.head()

,Cust_ID,Product_ID,Rating,Status_Dt
24,1375586,0,0.0,2015-05-28
25,1375586,1,0.0,2015-05-28
26,1375586,2,1.0,2015-05-28
27,1375586,3,0.0,2015-05-28
28,1375586,4,0.0,2015-05-28


In [3]:
csv_file = Path('Data/surprise_train.csv')
if not csv_file.is_file():
    print ('Creating Surprise Library Compliant File....')
    use_cols = ['Status_Dt', 'Cust_ID', 'Savings_Acct', 'Guarantees', 'Cur_Acct',\
           'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct',  \
            'Particular_Plus_Acct', 'Short_Term_Deposits', 'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds', \
            'Mortgage', 'Pension1', 'Loans', 'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2', \
               'Direct_Debit']
    train = pd.read_csv('Data/cleaned_train_ver2.csv', usecols = use_cols)
    create_surprise_lib_compliant_file(train, csv_file)
    del train
    gc.collect()
csv_file = Path('Data/surprise_subset_train.csv')
print('Loading train data from {}'.format(csv_file))
#I was running into memory issues so saved a subset of the file containing data
# from April to May 2015 & 2016
# csv_data = blaze.Data('Data/surprise_train.csv')
# # query = csv_data[csv_data['a'] > csv_data['b']]
# query = csv_data[csv_data['Status_Dt'].isin(['2015-04-28', '2015-05-28', '2016-04-28', '2016-05-28'])]
# train = pd.DataFrame.from_records(query, columns=query.fields)

# surprise_cols = ['Cust_ID', 'Product_ID', 'Rating', 'Status_Dt']
# train = pd.read_csv(csv_file, usecols = surprise_cols)
# train.drop(train[train.Rating.isnull()].index, inplace=True)
# train.to_csv('Data/surprise_subset_train.csv', index=False)
# train.head()
surprise_cols = ['Cust_ID', 'Product_ID', 'Rating', 'Status_Dt']
train = pd.read_csv(csv_file, usecols = surprise_cols)
train.head()

Loading train data from Data\surprise_subset_train.csv


,Cust_ID,Product_ID,Rating,Status_Dt
0,1375586,0,0.0,2015-05-28
1,1375586,1,0.0,2015-05-28
2,1375586,2,1.0,2015-05-28
3,1375586,3,0.0,2015-05-28
4,1375586,4,0.0,2015-05-28


In [4]:
train.isnull().sum()

Cust_ID       0
Product_ID    0
Rating        0
Status_Dt     0
dtype: int64

In [5]:
train["Mths_Since_2015"] = train["Status_Dt"].map(date_to_int).astype(np.int8)

In [6]:
train.Status_Dt.value_counts()

2016-05-28    22354872
2016-04-28    22278576
2015-05-28    15161358
Name: Status_Dt, dtype: int64

In [11]:
cust_ids = train[train.Status_Dt == '2016-05-28']['Cust_ID']

In [13]:
cust_ids = np.unique(cust_ids)

In [14]:
#use only 500000 cust_ids to train
cust_ids = cust_ids[0:500000]

In [16]:
train_subset = train[train.Cust_ID.isin(cust_ids)]

In [18]:
del train
gc.collect()

301

In [17]:
train_subset.shape

(33165974, 5)

# Split Train into Train and Validation set

In [20]:
#split train into train and validation set
test_date = '2016-05-28'
test_date_mths_since_2015 = date_to_int(test_date)
train_df = train_subset[train_subset.Mths_Since_2015 < test_date_mths_since_2015]
test_df = train_subset[train_subset.Mths_Since_2015 == test_date_mths_since_2015]
train_df.shape, test_df.shape

((21165974, 5), (12000000, 5))

In [21]:
del train_subset
gc.collect()

90

# Transform input train dataset into a format compliant with Surprise Library

In [22]:
# It is to specify how to read the dataframe.
reader = Reader(rating_scale=(0, 1))

# create the traindata from the dataframe...
train_data = Dataset.load_from_df(train_df[['Cust_ID', 'Product_ID', 'Rating']], reader)

# build the trainset from traindata to be compliant with surprise library's dataset format
trainset = train_data.build_full_trainset() 

In [23]:
#train_df will no longer be needed
del train_df
gc.collect()

0

In [24]:
testset = list(zip(test_df.Cust_ID.values, test_df.Product_ID.values, test_df.Rating.values))
testset[:3]

[(935046, 0, 0.0), (935046, 1, 0.0), (935046, 2, 1.0)]

# Define functions

In [25]:
def get_ratings(predictions):
    actual = np.array([pred.r_ui for pred in predictions])
    pred = np.array([pred.est for pred in predictions])
    
    return actual, pred

def get_errors(predictions, print_them=False):

    actual, pred = get_ratings(predictions)
    error = log_loss(actual, pred, eps=1e-15, labels=[0,1])
    return error
    
def run_surprise(algo, trainset, testset, verbose=True): 
    '''
        return train_dict, test_dict
    
        It returns two dictionaries, one for train and the other is for test
        Each of them have 3 key-value pairs, which specify ''rmse'', ''mape'', and ''predicted ratings''.
    '''
    start = datetime.now()
    # dictionaries that stores metrics for train and test..
    train = dict()
    test = dict()
    
    # train the algorithm with the trainset
    st = datetime.now()
    print('Training the model...')
    algo.fit(trainset)
    print('Done. time taken : {} \n'.format(datetime.now()-st))
    
    # ---------------- Evaluating train data--------------------#
    st = datetime.now()
    print('Evaluating the model with train data..')
    # get the train predictions (list of prediction class inside Surprise)
    build_testset = trainset.build_testset()
    
    train_preds = algo.test(build_testset)
    
    print('Train preds are {}'.format(train_preds))
    # get predicted ratings from the train predictions..
    train_actual_ratings, train_pred_ratings = get_ratings(train_preds)
    
    # get 'log_loss from the train predictions.
    train_log_loss = get_errors(train_preds)
    print('time taken : {}'.format(datetime.now()-st))
    
    if verbose:
        print('-'*15)
        print('Train Data')
        print('-'*15)
        print("LogLoss : {}".format(train_log_loss))
    
    #store them in the train dictionary
    if verbose:
        print('adding train results in the dictionary..')
    train['log_loss'] = train_log_loss
    train['predictions'] = train_pred_ratings
    
    #------------ Evaluating Test data---------------#
    st = datetime.now()
    print('\nEvaluating for test data...')
    # get the predictions( list of prediction classes) of test data
    test_preds = algo.test(testset)
    # get the predicted ratings from the list of predictions
    test_actual_ratings, test_pred_ratings = get_ratings(test_preds)
    # get error metrics from the predicted and actual ratings
    test_log_loss = get_errors(test_preds)
    print('time taken : {}'.format(datetime.now()-st))
    
    if verbose:
        print('-'*15)
        print('Test Data')
        print('-'*15)
        print("LogLoss : {}".format(test_log_loss))
    # store them in test dictionary
    if verbose:
        print('storing the test results in test dictionary...')
    test['log_loss'] = test_log_loss
    test['predictions'] = test_pred_ratings
    
    print('\n'+'-'*45)
    print('Total time taken to run this algorithm :', datetime.now() - start)
    
    # return two dictionaries train and test
    return train, test

In [26]:
products =  ['Savings_Acct', 'Guarantees', 'Cur_Acct', 'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct', \
             'Mas_Particular_Acct', 'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits', \
            'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1', 'Loans', 'Taxes', \
             'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit']
models_evaluation_train = dict()
models_evaluation_test = dict()

models_evaluation_train, models_evaluation_test

({}, {})

# Baseline Model

In [13]:
from surprise import BaselineOnly 

In [19]:
gc.collect()

0

In [17]:
# Specify options to compute user and item biases
bsl_options = {'method': 'sgd',
               'learning_rate': .001
               }
bsl_algo = BaselineOnly(bsl_options=bsl_options)
# run this algorithm.., It will return the train and test results..
bsl_train_results, bsl_test_results = run_surprise(bsl_algo, trainset, testset, verbose=True)
# train_preds = run_surprise(bsl_algo, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['bsl_algo'] = bsl_train_results 
models_evaluation_test['bsl_algo'] = bsl_test_results

Training the model...
Estimating biases using sgd...
Done. time taken : 0:01:53.582650 

Evaluating the model with train data..
time taken : 0:56:58.243653
---------------
Train Data
---------------
LogLoss : 0.3571407485363766
adding train results in the dictionary..

Evaluating for test data...
time taken : 0:09:31.325906
---------------
Test Data
---------------
LogLoss : 0.029859884499144507
storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 1:08:23.310751


<h1> Matrix Factorization Techniques </h1>
<h2> SVD Matrix Factorization User Product interactions </h2>

In [27]:
from surprise import SVD

In [28]:
# initialize the model
n_factors=100 # decrease the dimensionality to 100. q_i and p_u will be of 100 dimensions. This is a hyperparameter.
svd = SVD(n_factors=n_factors, biased=True, random_state=15, verbose=True)
svd_train_results, svd_test_results = run_surprise(svd, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['svd'] = svd_train_results 
models_evaluation_test['svd'] = svd_test_results

Training the model...
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Done. time taken : 0:16:49.960005 

Evaluating the model with train data..


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



time taken : 0:04:16.251591
---------------
Train Data
---------------
LogLoss : 0.38150360771575476
adding train results in the dictionary..

Evaluating for test data...
time taken : 0:03:42.971633
---------------
Test Data
---------------
LogLoss : 0.057762452544384744
storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 0:24:49.230125


<h2> SVD Matrix Factorization with implicit feedback from user ( user buying habits ) </h2>

In [29]:
from surprise import SVDpp

In [ ]:
# initialize the model
svdpp = SVDpp(n_factors=50, random_state=15, verbose=True)
svdpp_train_results, svdpp_test_results = run_surprise(svdpp, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['svdpp'] = svdpp_train_results 
models_evaluation_test['svdpp'] = svdpp_test_results

Training the model...
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
Done. time taken : 5:23:23.401136 

Evaluating the model with train data..


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



time taken : 0:16:31.618370
---------------
Train Data
---------------
LogLoss : 0.37011362952789095
adding train results in the dictionary..

Evaluating for test data...
